<a href="https://colab.research.google.com/github/zboraon/anthropocene_date/blob/main/anthropocene_changepoint_simulation_multiple_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary libraries



Install and load the necessary libraries.


In [ ]:
system("apt install -y jags")
system("apt install -y r-base")

In [ ]:
list.of.packages <- c("runjags", "mcp", "truncnorm", "coda", "dplyr", "ggplot2", "HDInterval")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, dependencies = TRUE)

In [ ]:
rm(list = ls())


In [ ]:
library("mcp")
library("runjags")
library("dplyr")
library("HDInterval")  # For HDI calculation
library("coda")

set.seed(1952)


# Simulating and tidying the data

In [ ]:
# mcp model
model = list(
  y ~ 1,
  1 ~ 0 + x
)

# Function to simulate dataset
simulate_dataset <- function() {
  # Generate random change point between 1948 and 1956
  cp <- rnorm(1, mean = 1952, sd = 1)
  cp <- pmax(pmin(cp, 1956), 1948)  # Ensure change point is within bounds

  # Generate random number of data points between 15 and 25
  n <- sample(15:25, 1)
  empty = mcp(model, sample = FALSE)
  # Simulate dataset
  df_sim <- data.frame(
    x = runif(n, 1930, 1970),
    ageerror = abs(rnorm(n, mean = 0, sd = 1))
  )
  df_sim$y <- empty$simulate(df_sim$x,
                             # Population-level:
                             int_1 =  0, x_2 = 0.25, cp_1 = cp, sigma = 0.25)

  # Add proxy error
  df_sim$proxyerror <- abs(rnorm(n, mean = 0, sd = 0.1))

  return(df_sim)
}

# Simulate 5 datasets
datasets <- replicate(5, simulate_dataset(), simplify = FALSE)

# Sort each dataset in the list by 'x' column
datasets_sorted <- lapply(datasets, function(df) {
  df[order(df$x), ]
})

# Add id column to each dataset
datasets_with_id <- Map(cbind, datasets_sorted, id = seq_along(datasets_sorted))

# Combine all datasets into a single list
combined_dataset <- do.call(rbind, datasets_with_id)

# Function to create id_name column
create_id_name <- function(id) {
  paste0("simulated_proxy_", id)
}

# Add id_name column
combined_dataset$id_name <- sapply(combined_dataset$id, create_id_name)

filtered_datatouse_selected_for_jags <- combined_dataset %>%
  select(x, ageerror, y, proxyerror, id) %>%
  rename(age = x, id = id, y = y, proxyerror = proxyerror, ageerror = ageerror) %>%
  mutate(proxyerror = ifelse(proxyerror == 0, proxyerror + 1e-10, proxyerror)) %>%
  mutate(ageerror = ifelse(ageerror == 0, ageerror + 1e-10, ageerror))

dataList_forjags <- as.list(filtered_datatouse_selected_for_jags)
dataList_forjags$minx <- min(filtered_datatouse_selected_for_jags$age)
dataList_forjags$maxx <- max(filtered_datatouse_selected_for_jags$age)

# The jags model

In [ ]:
filtered_datatouse_selected_for_jags <- combined_dataset %>%
  select(x, ageerror, y, proxyerror, id) %>%
  rename(age = x, id = id, y = y, proxyerror = proxyerror, ageerror = ageerror) %>%
  mutate(proxyerror = ifelse(proxyerror == 0, proxyerror + 1e-10, proxyerror)) %>%
  mutate(ageerror = ifelse(ageerror == 0, ageerror + 1e-10, ageerror))

dataList_forjags <- as.list(filtered_datatouse_selected_for_jags)
dataList_forjags$minx <- min(filtered_datatouse_selected_for_jags$age)
dataList_forjags$maxx <- max(filtered_datatouse_selected_for_jags$age)

modelString <- "
data {
  for (tr_ in 1:length(age)) {
  age_tr[tr_] <- age[tr_] / 1000
  ageerror_tr[tr_] <- ageerror[tr_] / 1000
  }
  MINX <- minx / 1000
  MAXX <- maxx / 1000
}
model {
  # Priors for population-level effects
  cp_1_tr ~ dt(mu_cp_1, 1/sigma_cp_1^2, nu_cp)T(MINX, MAXX)
  sigma_cp_1 ~ dt(0, 0.05^(-2), 1)T(0, )

  int_1 ~ dt(0, 5^(-2), 3)
  x_2_tr ~ dt(0, 100^(-2), 3)
  sigma_1 ~  dt(0, 5^(-2), 1)T(0, )
  mu_cp_1 ~ dunif(MINX,MAXX) #dnorm(1.950, 1^(-2))T(MINX, MAXX)

  nu_cp ~ dexp(1/30)

  # Priors for varying effects
  for (id_ in 1:max(id)) {
    cp_1_id_tr_uncentered[id_] ~ dt(0, 1/cp_1_sigma_tr^2,5) T(MINX - cp_1_tr, MAXX - cp_1_tr)
  }
  cp_1_id_tr <- cp_1_id_tr_uncentered - mean(cp_1_id_tr_uncentered)  # vectorized zero-centering
  cp_1_sigma_tr ~ dt(0, 0.05^(-2), 1)T(0, )

  # Model and likelihood
  for (i_ in 1:length(age_tr)) {
    X_1_[i_] <- min(trueage[i_], (cp_1_tr + cp_1_id_tr[id[i_]]))
    X_2_[i_] <- min(trueage[i_], MAXX) - (cp_1_tr + cp_1_id_tr[id[i_]])
    trueage[i_] ~ dunif(MINX,MAXX) #dnorm(0, 1^(-2))
    age_tr[i_] ~  dnorm(trueage[i_], 1 / ageerror_tr[i_]^2)

    # Fitted value
    y_[i_] =

      # Segment 1: y ~ 1 + x
      (trueage[i_] >= MINX) * int_1 +

      # Segment 2: y ~ 1 + (1 | id) ~ 0 + x
      (trueage[i_]>= (cp_1_tr + cp_1_id_tr[id[i_]])) * x_2_tr * X_2_[i_]

    # Fitted standard deviation
    tau_y_[i_] = (proxyerror[i_]^2 + (sigma_1)^2)^(-1)

    # Likelihood
    y[i_] ~  dnorm(y_[i_], tau_y_[i_]) # dnorm(y_[i_], tau_y_[i_])  # dt(y_[i_], tau_y_[i_], nu_y) #
  }
  # Transform to original scale:
  cp_1 <- cp_1_tr * 1000
  cp_1_id <- cp_1_id_tr * 1000
  cp_1_sd <- cp_1_sigma_tr * 1000
  x_2 <- x_2_tr / 1000
}
  "

sampleNO <- 20000
thinstepsNO <- 2
adaptNO <- 20000
burninNO <- 20000
nChains <- 3

writeLines(modelString, con="TEMPmodel.txt")
parameters <- c("cp_1", "cp_1_id", "cp_1_sd", "int_1", "sigma_1", "x_2", "cp_1_sd")
initsList <- list(cp_1_tr = 1.950
)

# run the model
runJagsOut_multiple_sim<- run.jags(method = "parallel",
                           model = "TEMPmodel.txt",
                           monitor = parameters,
                           data = dataList_forjags,
                           inits = initsList,
                           n.chains = nChains,
                           adapt = adaptNO,
                           burnin = burninNO,
                           sample = sampleNO,
                           thin = thinstepsNO,
                           summarise = FALSE,
                           plots = FALSE,
                           modules = 'glm')

In [ ]:
      codaSamples <- as.mcmc.list(runJagsOut_multiple_sim)
      save(runJagsOut_multiple_sim, file="anthropocene_cp_sim_MCMC.Rdata" )
      summary2023colabrun_multiple_sim <- summary(runJagsOut_multiple_sim)
      write.csv(summary2023colabrun_multiple_sim, "/content/summary_sim_runs_sim.csv")